In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pytrends.request import TrendReq
from pytrends import dailydata
import requests

from sklearn import linear_model
import statsmodels.api as sm

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
with open("words.txt") as f:
    words = f.readlines()
words = [x.strip() for x in words]
words = words[:5]
words

['hedge', 'dividend', 'earnings', 'inflation', 'markets']

In [4]:
def get_trend(key_word):
    trend = pd.DataFrame(dailydata.get_daily_data(key_word, 2020, 1, 2020, 6, wait_time = 0)[key_word])
    trend["change"] = trend.pct_change()
    return trend.dropna()

In [5]:
word_trends = {}    #dictionary with key=word, value=DataFrame with Google trend and pct change 
for word in words:
    word_trends[word] = get_trend(word)

hedge:2020-01-01 2020-01-31
hedge:2020-02-01 2020-02-29
hedge:2020-03-01 2020-03-31
hedge:2020-04-01 2020-04-30
hedge:2020-05-01 2020-05-31
hedge:2020-06-01 2020-06-30
dividend:2020-01-01 2020-01-31
dividend:2020-02-01 2020-02-29
dividend:2020-03-01 2020-03-31
dividend:2020-04-01 2020-04-30
dividend:2020-05-01 2020-05-31
dividend:2020-06-01 2020-06-30
earnings:2020-01-01 2020-01-31
earnings:2020-02-01 2020-02-29
earnings:2020-03-01 2020-03-31
earnings:2020-04-01 2020-04-30
earnings:2020-05-01 2020-05-31
earnings:2020-06-01 2020-06-30
inflation:2020-01-01 2020-01-31
inflation:2020-02-01 2020-02-29
inflation:2020-03-01 2020-03-31
inflation:2020-04-01 2020-04-30
inflation:2020-05-01 2020-05-31
inflation:2020-06-01 2020-06-30
markets:2020-01-01 2020-01-31
markets:2020-02-01 2020-02-29
markets:2020-03-01 2020-03-31
markets:2020-04-01 2020-04-30
markets:2020-05-01 2020-05-31
markets:2020-06-01 2020-06-30


In [6]:
word_trends["hedge"]

,hedge,change
date,,
2020-01-02,31.96,0.463370
2020-01-03,46.98,0.469962
2020-01-04,31.28,-0.334185
2020-01-05,25.65,-0.179987
2020-01-06,42.66,0.663158
...,...,...
2020-06-26,60.83,0.112676
2020-06-27,76.50,0.257603
2020-06-28,69.16,-0.095948


## Old Stuff

In [77]:
fortune_500 = pd.read_csv("Fortune 500 2017 - Fortune 500.csv")
fortune_500.head()

,Rank,Title,Website,Employees,Sector,Industry,Hqlocation,Hqaddr,Hqcity,Hqstate,Hqzip,Hqtel,Ceo,Ceo-title,Address,Ticker,Fullname,Revenues,Revchange,Profits,Prftchange,Assets,Totshequity
0,1,Walmart,http://www.walmart.com,2300000,Retailing,General Merchandisers,"Bentonville, AR",702 S.W. Eighth St.,Bentonville,AR,72716,479-273-4000,C. Douglas McMillon,"President, Chief Executive Officer & Director","702 S.W. Eighth St., Bentonville, AR 72716",WMT,"Wal-Mart Stores, Inc.",485873,0.8,13643.0,-7.2,198825,77798.0
1,2,Berkshire Hathaway,http://www.berkshirehathaway.com,367700,Financials,Insurance: Property and Casualty (Stock),"Omaha, NE",3555 Farnam St.,Omaha,NE,68131,402-346-1400,Warren E. Buffett,Chairman & Chief Executive Officer,"3555 Farnam St., Omaha, NE 68131",BRKA,Berkshire Hathaway Inc.,223604,6.1,24074.0,0,620854,283001.0
2,3,Apple,http://www.apple.com,116000,Technology,"Computers, Office Equipment","Cupertino, CA",1 Infinite Loop,Cupertino,CA,95014,408-996-1010,Timothy D. Cook,Chief Executive Officer & Director,"1 Infinite Loop, Cupertino, CA 95014",AAPL,"Apple, Inc.",215639,-7.7,45687.0,-14.4,321686,128249.0
3,4,Exxon Mobil,http://www.exxonmobil.com,72700,Energy,Petroleum Refining,"Irving, TX",5959 Las Colinas Blvd.,Irving,TX,75039,972-444-1000,Darren W. Woods,Chairman & Chief Executive Officer,"5959 Las Colinas Blvd., Irving, TX 75039",XOM,Exxon Mobil Corporation,205004,-16.7,7840.0,-51.5,330314,167325.0
4,5,McKesson,http://www.mckesson.com,68000,Wholesalers,Wholesalers: Health Care,"San Francisco, CA",1 Post St.,San Francisco,CA,94104,415-983-8300,John H. Hammergren,"Chairman, President & Chief Executive Officer","1 Post St., San Francisco, CA 94104",MCK,McKesson Corporation,192487,6.2,2258.0,53,56563,8924.0


In [13]:
def get_name(symbol):
    url = "http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={}&region=1&lang=en".format(symbol)

    result = requests.get(url).json()

    for x in result['ResultSet']['Result']:
        if x['symbol'] == symbol:
            return x['name']

Microsoft Corporation


In [116]:
def get_data(ticker: str):
    print("Stock data for: \"" + str(ticker) + "\"")
    data = pd.DataFrame(yf.download(ticker, start = '2020-1-1', end = "2020-6-1", interval = "1d")["Adj Close"])
    name = fortune_500[fortune_500["Ticker"] == ticker][["Title"]].iloc[0][0]
    print("Search trends for: \"" + str(name) + "\"")
    trend = dailydata.get_daily_data(name, 2020, 1, 2020, 6, wait_time = 0)[name]
    data.index.name = "date"
    data = data.join(trend, how = "inner")
    return data

In [117]:
df = get_data("TSLA")
df.head(50)

Stock data for: "TSLA"
[*********************100%***********************]  1 of 1 completed
Search trends for: "Tesla"
Tesla:2020-01-01 2020-01-31
Tesla:2020-02-01 2020-02-29
Tesla:2020-03-01 2020-03-31
Tesla:2020-04-01 2020-04-30
Tesla:2020-05-01 2020-05-31
Tesla:2020-06-01 2020-06-30


,Adj Close,Tesla
date,,
2020-01-02,86.052002,21.96
2020-01-03,88.601997,23.94
2020-01-06,90.307999,15.60
2020-01-07,93.811996,22.94
2020-01-08,98.428001,23.18
2020-01-09,96.267998,23.68
2020-01-10,95.629997,19.14
2020-01-13,104.972000,24.32
2020-01-14,107.584000,27.20
